In [ ]:
pip install mediapipe opencv-python numpy

In [2]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Function to handle alerts
def send_alert_to_emergency_services():
    print("Alert: Emergency services contacted!")

# Function to detect SOS gesture
def detect_sos_gesture(landmarks, state, time_in_state):
    # Define key landmarks for simplicity
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    thumb_ip = landmarks[3]  # Thumb interphalangeal joint

    # Open palm: all fingers extended
    is_open_palm = (index_tip.y < landmarks[6].y and
                    middle_tip.y < landmarks[10].y and
                    ring_tip.y < landmarks[14].y and
                    pinky_tip.y < landmarks[18].y and
                    thumb_tip.x > thumb_ip.x)

    # Fist: all fingers closed over thumb
    is_fist = (index_tip.y > landmarks[6].y and
               middle_tip.y > landmarks[10].y and
               ring_tip.y > landmarks[14].y and
               pinky_tip.y > landmarks[18].y)

    # State transitions with time consistency
    current_time = time.time()
    if state == 0 and is_open_palm:
        print("Step 1: Open palm detected.")
        return 1, current_time  # Move to next step
    elif state == 1 and (current_time - time_in_state > 0.5):
        print("Step 2: Waiting for thumb tucked.")
        return 2, current_time  # Move to next step
    elif state == 2 and is_fist and (current_time - time_in_state > 0.5):
        print("Step 3: Fist made detected.")
        return 3, current_time  # Full SOS gesture recognized

    return state, time_in_state

# Load video from file
video_path = "diya_sos.mp4"  # Path to the video file
cap = cv2.VideoCapture(video_path)

# Initial gesture state and time tracking
gesture_state = 0
state_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with the hand recognition model
    result = hands.process(rgb_frame)

    # Check for detected hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks for visual confirmation
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

            # Extract landmark positions
            landmarks = hand_landmarks.landmark
            
            # Check for SOS gesture with state management
            gesture_state, state_time = detect_sos_gesture(landmarks, gesture_state, state_time)

            # Reset state if gesture sequence completed
            if gesture_state == 3:
                send_alert_to_emergency_services()
                gesture_state = 0  # Reset after alert

    else:
        # print("No hand detected in the current frame.")
        gesture_state = 0  # Reset if no hand is detected

    # Show the frame for visual confirmation
    cv2.imshow('Hand Gesture Recognition', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

Step 1: Open palm detected.
Step 2: Waiting for thumb tucked.
Step 3: Fist made detected.
Alert: Emergency services contacted!


### Output getting saved as mp4 file

In [6]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Function to handle alerts
def send_alert_to_emergency_services():
    print("Alert: Emergency services contacted!")

# Function to detect SOS gesture
def detect_sos_gesture(landmarks, state, time_in_state):
    # Define key landmarks for simplicity
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    thumb_ip = landmarks[3]  # Thumb interphalangeal joint

    # Open palm: all fingers extended
    is_open_palm = (index_tip.y < landmarks[6].y and
                    middle_tip.y < landmarks[10].y and
                    ring_tip.y < landmarks[14].y and
                    pinky_tip.y < landmarks[18].y and
                    thumb_tip.x > thumb_ip.x)

    # Fist: all fingers closed over thumb
    is_fist = (index_tip.y > landmarks[6].y and
               middle_tip.y > landmarks[10].y and
               ring_tip.y > landmarks[14].y and
               pinky_tip.y > landmarks[18].y)

    # State transitions with time consistency
    current_time = time.time()
    if state == 0 and is_open_palm:
        print("Step 1: Open palm detected.")
        return 1, current_time  # Move to next step
    elif state == 1 and (current_time - time_in_state > 0.5):
        print("Step 2: Waiting for thumb tucked.")
        return 2, current_time  # Move to next step
    elif state == 2 and is_fist and (current_time - time_in_state > 0.5):
        print("Step 3: Fist made detected.")
        return 3, current_time  # Full SOS gesture recognized

    return state, time_in_state

# Load video from file
video_path = "diya_sos.mp4"  # Path to the video file
cap = cv2.VideoCapture(video_path)

# Get video details for saving output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter('output_sos_detection.mp4', fourcc, fps, (frame_width, frame_height))

# Initial gesture state and time tracking
gesture_state = 0
state_time = time.time()
sos_detected = False  # Variable to track if SOS gesture has been detected

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with the hand recognition model
    result = hands.process(rgb_frame)

    # Check for detected hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks for visual confirmation
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

            # Extract landmark positions
            landmarks = hand_landmarks.landmark
            
            # Check for SOS gesture with state management if not already detected
            if not sos_detected:
                gesture_state, state_time = detect_sos_gesture(landmarks, gesture_state, state_time)

                # If SOS gesture is detected, send alert and mark SOS as detected
                if gesture_state == 3:
                    send_alert_to_emergency_services()
                    sos_detected = True  # Mark that SOS gesture has been detected
                    gesture_state = 0  # Reset state (not necessary anymore)
    
    # Annotate frame if SOS gesture is detected
    if sos_detected:
        cv2.putText(frame, "SOS Gesture Detected!", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.putText(frame, "Emergency Services Contacted", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Write the frame to the output video file
    out.write(frame)

    # Show the frame for visual confirmation (optional)
    cv2.imshow('Hand Gesture Recognition', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Release the VideoWriter object
cv2.destroyAllWindows()

Step 1: Open palm detected.
Step 2: Waiting for thumb tucked.
Step 3: Fist made detected.
Alert: Emergency services contacted!
